# Tokenisation

Processus de segmentation d’un texte en unités (tokens) significatives (phrases, mots, symboles…).

Souvent la première étape avant toute analyse de texte.

## Préparation

1. Importer NLTK
2. Installer le module `punkt` (si besoin)
3. Charger le tokenizer

In [ ]:
import nltk
#nltk.download()
from nltk.tokenize import sent_tokenize, word_tokenize

## Segmentation en phrases

Méthode `sent_tokenize()`

Exemple avec deux extraits issus des *Adventures of Buster Bear* (Burgess, 1920) et du *Cid* (Corneille, 1637) :

In [ ]:
en = "Little Joe sat down on the bank and prepared to enjoy his breakfast. He hadn't seen Buster Bear, and he didn't know that he or any one else was anywhere near."
fr = "Elvire, m’as-tu fait un rapport bien sincère ? Ne déguises-tu rien de ce qu’a dit mon père ?"
sent_tokenize(en)

Pour plus d’efficacité avec des textes longs, charger directement le fichier *pickle* (outil pour sérialiser un objet en Python)

Et sélectionner celui qui convient à la langue de travail :

In [ ]:
import nltk.data
tokenizer = nltk.data.load('tokenizers/punkt/PY3/french.pickle')
tokenizer.tokenize(fr)

## Segmentation en mots

Méthode `word_tokenize()` :

In [ ]:
print(word_tokenize(en))
print(word_tokenize(fr, language='french'))

Remarques :
- ponctuation conservée
- contractions séparées (en anglais)

Segmentation selon espaces et ponctuation.

Méthode `word_tokenize()` fait par défaut appel à une classe `TreebankWordTokenizer`.

D’autres *tokenizers* existent :
- `WhitespaceTokenizer`
- `SpaceTokenizer`
- `WordPunctTokenizer`

Comparons les différences :

In [ ]:
from nltk.tokenize import WhitespaceTokenizer, SpaceTokenizer, WordPunctTokenizer
sentence = "He hadn't seen Buster Bear, and he didn't know that he or any one else was anywhere near."
tokenizers = [WhitespaceTokenizer(), SpaceTokenizer(), WordPunctTokenizer()]
print([tokenizer.tokenize(sentence) for tokenizer in tokenizers])

Les contractions et la ponctuation sont gérées différemment :

| Tokenizer    | Contraction      | Ponctuation |
|--------------|:----------------:|:------------|
| TreebankWord | *had*, *n't*     | excluse     |
| Whitespace   | *hadn't*         | incluse     |
| Space        | *hadn't*         | incluse     |
| WordPunct    | *hadn*, *'*, *t* | excluse     |

Une autre classe de tokenizer permet de personnaliser son modèle de tokenisation : `RegexpTokenizer`

Utile par exemple pour supprimer toute ponctuation :

In [ ]:
from nltk.tokenize import RegexpTokenizer
tokenizer = RegexpTokenizer('\w+')
print(tokenizer.tokenize(fr))

Et pour inclure les contractions de l’anglais ?

In [ ]:
from nltk.tokenize import RegexpTokenizer
tokenizer = RegexpTokenizer("[\w']+")
print(tokenizer.tokenize(en))

Par défaut, `RegexpTokenizer` travaille sur les tokens, mais on peut lui demander de travailler plutôt sur les espaces :

In [ ]:
from nltk.tokenize import RegexpTokenizer
tokenizer = RegexpTokenizer("[\s]+", gaps=True)
print(tokenizer.tokenize(en))

Une autre opération courante consiste à ne conserver que les mots signifiants d’un texte.

NLTK fournit une liste de mots vides pour plusieurs langues :

In [ ]:
from nltk.corpus import stopwords
# Sample of stopwords in French
print(stopwords.words('french')[::4])

Et pour filtrer grâce à cette liste :

In [ ]:
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
tokenizer = RegexpTokenizer("\w+")
fr_stopwords = stopwords.words('french')
words = tokenizer.tokenize(fr)
print([word for word in words if word not in fr_stopwords])

## Application

Affichez une liste triée par fréquence des mots présents dans *Moby Dick* de Herman Melville.

**Note :** le texte est disponible dans le corpus Gutenberg de NLTK.

In [7]:
import nltk, unicodedata, string
from nltk.corpus import gutenberg, stopwords

def remove_accent_mark(word):
    """Removes the accent marks on a character.
    
    Argument keyword:
    word -- the word to analyse
    """
    word_norm = ""
    for character in word:
        char_norm = unicodedata.normalize('NFKD', character)
        word_norm += char_norm[0]
    return word_norm

words = gutenberg.words('melville-moby_dick.txt')
occurrences = {}
for word in words:
    if word not in stopwords.words('english') + list(string.punctuation):
        occurrences.update({
            word: occurrences.get(word, 0) + 1
        })

words_sorted = list(occurrences.items())
words_sorted = sorted(words_sorted, key=lambda this:remove_accent_mark(this[0].lower()))
words_sorted = sorted(words_sorted, key=lambda this:this[1], reverse=True)